In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.ensemble import RandomForestClassifier, IsolationForest
from sklearn.metrics import accuracy_score
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
import os
import pickle
import json

# --- 0. Setup and File Management ---
def clear_stale_model_files():
    """Deletes old model and preprocessing files to force retraining."""
    print("--- Clearing stale model files ---")
    files_to_delete = [
        'ann_model.pth', 'rf_model.pkl', 'iso_forest_model.pkl', 
        'scaler.pkl', 'feature_columns.pkl', 'label_encoders.pkl'
    ]
    for f in files_to_delete:
        if os.path.exists(f):
            os.remove(f)

def create_default_policy():
    """Creates a default policy.json file if one doesn't exist."""
    if not os.path.exists('policy.json'):
        default_policy = {
          "LowConfidenceThreat": { "confidence_threshold": 0.50, "priority_actions": ["MONITOR", "LOG_EVENT"] },
          "MediumConfidenceThreat": { "confidence_threshold": 0.75, "priority_actions": ["FLAG_FOR_REVIEW", "MONITOR"] },
          "HighConfidenceThreat": { "confidence_threshold": 0.90, "priority_actions": ["BLOCK_IP", "ISOLATE_HOST"] },
          "KnownThreatIoC": { "confidence_threshold": 0.99, "priority_actions": ["BLOCK_IP", "ISOLATE_HOST", "RUN_SCAN"] }
        }
        with open('policy.json', 'w') as f:
            json.dump(default_policy, f, indent=2)
        print("--- Created default policy.json ---")

# --- 1. Data Loading and Preprocessing ---
def preprocess_data(net_path='network_traffic_logs.csv', sys_path='system_event_logs.csv', intel_path='threat_intelligence.csv'):
    """Loads, combines, and preprocesses all data sources including threat intelligence."""
    print("--- Starting Data Preprocessing ---")
    df_net = pd.read_csv(net_path)
    df_sys = pd.read_csv(sys_path)
    df_intel = pd.read_csv(intel_path)

    known_threat_ips = set(df_intel[df_intel['Threat Type'] != 'Phishing']['IOC'])
    
    df_net['is_known_threat_ip'] = df_net.apply(
        lambda row: 1 if row['Source IP'] in known_threat_ips or row['Destination IP'] in known_threat_ips else 0, axis=1
    )
    
    df_net['Timestamp'] = pd.to_datetime(df_net['Timestamp'])
    df_sys['Timestamp'] = pd.to_datetime(df_sys['Timestamp'])
    df_sys.sort_values('Timestamp', inplace=True)
    
    lookback_window = pd.Timedelta(hours=10)
    aggregated_features = []

    for index, net_row in df_net.iterrows():
        end_time = net_row['Timestamp']
        start_time = end_time - lookback_window
        recent_sys_events = df_sys[(df_sys['Timestamp'] > start_time) & (df_sys['Timestamp'] <= end_time)]
        
        features = {
            'timestamp': end_time,
            'recent_failed_logins': (recent_sys_events['Event Type'] == 'Failed Login').sum(),
            'recent_priv_escalations': (recent_sys_events['Event Type'] == 'Privilege Escalation').sum(),
            'is_system_threat_present': 1 if 'Yes' in recent_sys_events['Threat'].unique() else 0
        }
        aggregated_features.append(features)

    df_agg_sys = pd.DataFrame(aggregated_features)
    df = pd.merge(df_net, df_agg_sys, left_on='Timestamp', right_on='timestamp', how='left').fillna(0)
    
    original_df = df.copy()

    df['Threat'] = LabelEncoder().fit_transform(df['Threat'])
    df['Combined_Threat'] = (df['Threat'] | df['is_system_threat_present'] | df['is_known_threat_ip']).astype(int)

    df_features = df.drop(['Timestamp', 'Source IP', 'Destination IP', 'Threat Type', 'Threat', 'timestamp', 'is_system_threat_present', 'Combined_Threat'], axis=1)
    
    encoders = {}
    for col in ['Protocol']:
        le = LabelEncoder()
        df_features[col] = le.fit_transform(df_features[col].astype(str))
        encoders[col] = le
        
    X = df_features
    y = df['Combined_Threat']
    
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    
    print("--- Data Preprocessing Complete ---\n")
    return X_scaled, y.values, original_df, scaler, X.columns, encoders

# --- 2. PyTorch Model Definition and Training ---
class ANN_PyTorch(nn.Module):
    def __init__(self, input_dim):
        super(ANN_PyTorch, self).__init__()
        self.layer_1 = nn.Linear(input_dim, 128)
        self.layer_2 = nn.Linear(128, 64)
        self.layer_3 = nn.Linear(64, 32)
        self.layer_out = nn.Linear(32, 1)
        
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=0.5)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.relu(self.layer_1(x))
        x = self.dropout(x)
        x = self.relu(self.layer_2(x))
        x = self.dropout(x)
        x = self.relu(self.layer_3(x))
        x = self.sigmoid(self.layer_out(x))
        return x

def train_pytorch_model(X_train, y_train, X_test, y_test):
    """Defines the training loop for the PyTorch ANN model."""
    # Convert data to PyTorch Tensors
    X_train_tensor = torch.FloatTensor(X_train)
    y_train_tensor = torch.FloatTensor(y_train).view(-1, 1)
    X_test_tensor = torch.FloatTensor(X_test)
    y_test_tensor = torch.FloatTensor(y_test).view(-1, 1)

    # Create DataLoader
    train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
    train_loader = DataLoader(dataset=train_dataset, batch_size=32, shuffle=True)

    # Initialize model, loss, and optimizer
    input_dim = X_train.shape[1]
    model = ANN_PyTorch(input_dim)
    criterion = nn.BCELoss() # Binary Cross-Entropy Loss for binary classification
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    # Training loop
    epochs = 50
    for epoch in range(epochs):
        model.train()
        for X_batch, y_batch in train_loader:
            optimizer.zero_grad()
            y_pred = model(X_batch)
            loss = criterion(y_pred, y_batch)
            loss.backward()
            optimizer.step()

    # Evaluation
    model.eval()
    with torch.no_grad():
        y_pred_tensor = model(X_test_tensor)
        y_pred = (y_pred_tensor > 0.5).float()
        accuracy = accuracy_score(y_test_tensor, y_pred)
        print(f"PyTorch ANN Test Accuracy: {accuracy*100:.2f}%")

    return model

def train_models(X, y, scaler, encoders):
    """Trains and saves the hybrid detection models."""
    print("\n--- Training Hybrid Detection Models ---")
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, shuffle=True, stratify=y)

    # a) Supervised: Random Forest
    rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
    rf_model.fit(X_train, y_train)
    print(f"Random Forest Test Accuracy: {rf_model.score(X_test, y_test)*100:.2f}%")

    # b) Supervised: PyTorch ANN
    ann_model = train_pytorch_model(X_train, y_train, X_test, y_test)
    
    # c) Unsupervised: Isolation Forest
    iso_forest_model = IsolationForest(contamination='auto', random_state=42)
    iso_forest_model.fit(X_train)

    # Save models and preprocessing objects
    torch.save(ann_model.state_dict(), 'ann_model.pth')
    with open('rf_model.pkl', 'wb') as f: pickle.dump(rf_model, f)
    with open('iso_forest_model.pkl', 'wb') as f: pickle.dump(iso_forest_model, f)
    with open('scaler.pkl', 'wb') as f: pickle.dump(scaler, f)
    with open('label_encoders.pkl', 'wb') as f: pickle.dump(encoders, f)
    print("--- All Models and Preprocessing Objects Saved ---")

# --- 3. Multi-Agent System ---
class DetectionAgent:
    """Uses a hybrid model approach for threat detection."""
    def __init__(self, feature_columns):
        print("--- Initializing Detection Agent ---")
        input_dim = len(feature_columns)
        self.ann_model = ANN_PyTorch(input_dim)
        self.ann_model.load_state_dict(torch.load('ann_model.pth'))
        self.ann_model.eval() # Set model to evaluation mode

        with open('rf_model.pkl', 'rb') as f: self.rf_model = pickle.load(f)
        with open('iso_forest_model.pkl', 'rb') as f: self.iso_forest_model = pickle.load(f)
        with open('scaler.pkl', 'rb') as f: self.scaler = pickle.load(f)
        with open('label_encoders.pkl', 'rb') as f: self.encoders = pickle.load(f)
        self.feature_columns = feature_columns

    def analyze(self, new_data):
        print("\n>>> [Detection Agent] Analyzing new data with hybrid model...")
        data_df = pd.DataFrame([new_data])
        
        data_processed = data_df.copy()
        for col, encoder in self.encoders.items():
            data_processed[col] = encoder.transform(data_processed[col].astype(str))
        
        data_processed = data_processed[self.feature_columns]
        data_scaled = self.scaler.transform(data_processed)
        
        # PyTorch prediction
        with torch.no_grad():
            data_tensor = torch.FloatTensor(data_scaled)
            ann_pred_prob = self.ann_model(data_tensor).item()

        rf_probs = self.rf_model.predict_proba(data_scaled)
        rf_pred_prob = rf_probs[0][1] if rf_probs.shape[1] > 1 else (1.0 if self.rf_model.classes_[0] == 1 else 0.0)

        iso_pred = self.iso_forest_model.predict(data_scaled)[0]
        
        anomaly_score = 1 if iso_pred == -1 else 0
        final_confidence = (ann_pred_prob + rf_pred_prob + anomaly_score) / 3.0
        
        alert = {
            'is_threat': bool(final_confidence > 0.5),
            'is_known_threat': bool(data_processed['is_known_threat_ip'].iloc[0] == 1),
            'confidence': f"{final_confidence:.2%}",
            'details': data_df.to_dict('records')[0]
        }
        print(f">>> [Detection Agent] Analysis complete. Threat Detected: {alert['is_threat']} (Confidence: {alert['confidence']})")
        return alert

class ResponseAgent:
    """Acts based on a dynamic, file-based policy."""
    def __init__(self, policy_file='policy.json'):
        print("--- Initializing Response Agent ---")
        with open(policy_file, 'r') as f:
            self.policy = json.load(f)

    def formulate_response(self, alert):
        print(">>> [Response Agent] Consulting policy...")
        confidence = float(alert['confidence'].strip('%')) / 100
        
        if alert['is_known_threat']:
            threat_level = "KnownThreatIoC"
        elif confidence > self.policy['HighConfidenceThreat']['confidence_threshold']:
            threat_level = "HighConfidenceThreat"
        elif confidence > self.policy['MediumConfidenceThreat']['confidence_threshold']:
            threat_level = "MediumConfidenceThreat"
        elif confidence > self.policy['LowConfidenceThreat']['confidence_threshold']:
            threat_level = "LowConfidenceThreat"
        else:
            return "No action required. Continue monitoring."
        
        actions = self.policy[threat_level]['priority_actions']
        print(f">>> [Response Agent] Threat Level '{threat_level}'. Executing actions: {actions}")
        return actions

class LearningAgent:
    """RL-based agent to adapt and improve the system."""
    def __init__(self, log_file='incident_log.csv'):
        print("--- Initializing Learning Agent (RL Core) ---")
        self.log_file = log_file

    def get_feedback_and_reward(self, alert, action):
        """Simulates getting feedback and calculating a reward."""
        print(">>> [Learning Agent] Analyzing action results...")
        confidence = float(alert['confidence'].strip('%')) / 100
        is_correct_action = ('BLOCK_IP' in action and confidence > 0.8) or ('MONITOR' in action and confidence < 0.8)
        
        reward = 1 if is_correct_action else -1
        print(f">>> [Learning Agent] Action effectiveness assessed. Reward: {reward}")
        return reward

    def update_policy(self, reward):
        """Placeholder for the DQN logic to update the Response Agent's policy."""
        print(">>> [Learning Agent] Policy refinement signal sent (Reward: {}).".format(reward))
        pass
        
    def update_models(self, alert, action):
        """Placeholder for updating Detection Agent models."""
        print(">>> [Learning Agent] Model update signal sent.")
        pass

# --- 4. Main Execution ---
def main():
    """Main function to run the full multi-agent system simulation."""
    clear_stale_model_files()
    create_default_policy()
    
    # --- Training Phase ---
    X_scaled, y, original_df, scaler, feature_columns, encoders = preprocess_data()
    with open('feature_columns.pkl', 'wb') as f: pickle.dump(feature_columns, f)
    train_models(X_scaled, y, scaler, encoders)
    
    # --- Operational Phase ---
    print("\n" + "="*50)
    print("--- Initializing Multi-Agent Cybersecurity Watchdog ---")
    detection_agent = DetectionAgent(feature_columns)
    response_agent = ResponseAgent()
    learning_agent = LearningAgent()

    # --- Simulation Loop ---
    print("\n" + "="*50)
    print("--- Simulating New Incoming Data Event ---")
    
    new_event_data = {
        'Packet Size (bytes)': 1490.0, 'Flow Duration (s)': 0.2, 'Protocol': 'TCP',
        'is_known_threat_ip': 0, 'recent_failed_logins': 8, 'recent_priv_escalations': 2,
        'Source IP': '198.51.100.23', 'Destination IP': '10.0.0.5'
    }
    
    alert = detection_agent.analyze(new_event_data)
    actions = response_agent.formulate_response(alert)
    reward = learning_agent.get_feedback_and_reward(alert, actions)
    learning_agent.update_policy(reward)
    learning_agent.update_models(alert, actions)
    
    print("\n--- System Cycle Complete ---")


if __name__ == "__main__":
    # Set random seed for reproducibility
    np.random.seed(42)
    torch.manual_seed(42)
    main()

--- Clearing stale model files ---
--- Starting Data Preprocessing ---


FileNotFoundError: [Errno 2] No such file or directory: 'network_traffic_logs.csv'